In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Import the Data

In [ ]:
path = '/home/alvaro/tf_templates/DATA/Quijote.txt'

with open(path, 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenize the Text

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

## Preprocess Input and Output Data

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

X, y = input_sequences[:, :-1], input_sequences[:, -1]

## Create the Model

In [ ]:
model = tf.keras.Sequential()
model.add(Embedding(total_words, 250, input_length=max_sequence_length-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=30, verbose=1)

## Save

In [ ]:
model.save('/home/alvaro/tf_templates/NLP/quijote.keras')

In [ ]:
model = load_model('/home/alvaro/tf_templates/NLP/quijote.keras')

## Generate New Text

In [ ]:
seed_text = "En un lugar de la Mancha"
next_words = 1000

for w in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list, verbose=0), axis=1)
    output_word = tokenizer.index_word.get(predicted[0], '')
    seed_text += " " + output_word

print(seed_text)